In [1]:
import sys 

sys.path.insert(0, '..')

In [13]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

In [14]:
image_path = './'

celeba_train_dataset = torchvision.datasets.CelebA(image_path, split='train', target_type='attr', download=False)
celeba_valid_dataset = torchvision.datasets.CelebA(image_path, split='valid', target_type='attr', download=False)
celeba_test_dataset = torchvision.datasets.CelebA(image_path, split='test', target_type='attr', download=False)

print('Train set:', len(celeba_train_dataset))
print('Validation set:', len(celeba_valid_dataset))
print('Test set:', len(celeba_test_dataset))

Train set: 162770
Validation set: 19867
Test set: 19962


In [15]:
# transformers to augment data to produce more trainable data

get_smile = lambda attr: attr[31]

transform_train = transforms.Compose([
    transforms.RandomCrop([178, 178]),
    transforms.RandomHorizontalFlip(),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.CenterCrop([178, 178]),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
])

In [16]:
torch.manual_seed(1)

celeba_train_dataset = torchvision.datasets.CelebA(image_path, split='train', target_type='attr', download=False, transform=transform_train, target_transform=get_smile)
celeba_valid_dataset = torchvision.datasets.CelebA(image_path, split='valid', target_type='attr', download=False, transform=transform_test, target_transform=get_smile)
celeba_test_dataset = torchvision.datasets.CelebA(image_path, split='test', target_type='attr', download=False, transform=transform_test, target_transform=get_smile)

data_loader = DataLoader(celeba_train_dataset, batch_size=2)

In [ ]:

# train on small dataset as a challenge
celeba_train_dataset = Subset(celeba_train_dataset, torch.arange(20000))
celeba_valid_dataset = Subset(celeba_valid_dataset, torch.arange(1000)) 

In [18]:
batch_size = 32

train_dl = DataLoader(celeba_train_dataset, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(celeba_valid_dataset, batch_size, shuffle=False)
test_dl = DataLoader(celeba_test_dataset, batch_size, shuffle=False)